In [1]:
import openpharmacophore.pharmacophore.pl_interactions as pli
import mdtraj as mdt
import nglview as nv
import numpy as np
from matplotlib.colors import to_rgb

# Visualize the binding site of protein-ligand complex

In [2]:
def extract_chain(traj, chains):
    """ Extract chains from a trajectory

        Parameters
        ----------
        traj: mdtraj.trajectory

        chains: list[int]

        Returns
        -------
        new_traj: mdtraj.Trajectory

    """
    topology = traj.topology
    return traj.atom_slice(
        [atom.index for atom in topology.atoms if (atom.residue.chain.index in chains)]
    )

In [3]:
def add_sphere_to_view(view, center, radius):
    """ Add a sphere to a view. """
    color = '#B03A2E'
    n_components = len(view._ngl_component_ids)
    view.shape.add_sphere(center.tolist(), to_rgb(color), radius, "sphere")
    view.update_representation(component=n_components, repr_index=0, opacity=0.3)

In [4]:
traj = mdt.load("../../test_cases/eralpha/1qku/1qku.pdb")
traj = extract_chain(traj, [0, 3])
view = nv.show_mdtraj(traj)

In [5]:
traj.save_pdb("er_alpha_A_chain.pdb")

In [6]:
print(pli.find_ligands_in_traj(traj))

['EST:B']


In [7]:
ligand_id = "EST:B"
centroid = pli.ligand_centroid(traj, ligand_id) * 10 # convert to angstroms
extent = pli.ligand_maximum_extent(traj, ligand_id) * 10

In [8]:
bs_site = extent + 8.5

In [9]:
add_sphere_to_view(view, centroid, bs_site)
view.representations = [
     {"type": "ball+stick", 
      "params": {
          "sele": "protein",
          "color": "residueindex"
      }},
    {"type": "ball+stick", 
     "params": {
         "sele": "( not polymer or hetero ) and not ( water or ion )"
    }}
]
view

NGLWidget()

## Extract binding site
Extract binding site to a new trajectory for visualization purposes

In [10]:
BS_DIST = 8.5  # in angstroms


def get_binding_site_atoms_indices(lig_center, lig_max_extent, coords):
    """ Get the indices of all the atoms that belong to the binding site.

        Parameters
        ----------
        lig_center : np.array of shape (3,)
        lig_max_extent : float
        coords: np.array of shape(n_atoms, 3)

        Returns
        -------
        bs_indices: list[int]
    """
    bs_cutoff = lig_max_extent + BS_DIST
    distance = np.sqrt(np.sum(np.power(coords - lig_center, 2), axis=1))
    return np.where(distance <= bs_cutoff)[0]

In [11]:
bs_indices = get_binding_site_atoms_indices(centroid, extent, traj.xyz[0] * 10)
bs_indices.shape

(656,)

In [12]:
bs_traj = traj.atom_slice(bs_indices)
print(bs_traj)

<mdtraj.Trajectory with 1 frames, 656 atoms, 106 residues, and unitcells>


In [13]:
bs_view = nv.show_mdtraj(bs_traj)
bs_view.representations = [
     {"type": "ball+stick", 
      "params": {
          "sele": "protein",
      }},
    {"type": "ball+stick", 
     "params": {
         "sele": "( not polymer or hetero ) and not ( water or ion )",
         "color": "blue"
    }}
]
bs_view

NGLWidget()